In [ ]:
######################################################################
## 02_ANALYSIS: Preparation of BSseq Objects
## (WGBS prep + the WGS steps needed to build SNP masks for BSseq)
######################################################################

# NOTE: Blocks labeled “SCRIPT / ON COMMAND LINE” are terminal/SLURM steps, not R.

# Prerequisites (what should already exist before BSseq masking):
# - WGBS reads trimmed and mapped with Bismark (deduplicated .cov.gz files present)
# - Reference FASTA indexed: mytilus_californianus_genome.fasta(.fai)
# - WGS Bowtie2 BAMs: ~/Mytilus-WGS/mapping_bt2/*.mkdup.bam + .bai
# - FreeBayes relaxed per-sample VCFs merged to a cohort VCF (built below) which we then convert to SNP masks

In [1]:
######################################################################
## BLOCK 0A — Global paths (run in Notebook)
######################################################################

proj_dir    <- "/home/local/ADS/lsh/Mytilus-WGS"
ref_fa      <- "/home/local/ADS/lsh/Mytilus/prepped_genome/mytilus_californianus_genome.fasta"

# WGS (Bowtie2) mapping outputs (mkdup BAMs)
map_bt2_dir <- file.path(proj_dir, "mapping_bt2")

# Optional: mosdepth coverage outputs
cov_bt2_dir <- file.path(proj_dir, "wgs_coverage_bt2")

# SNP/mask outputs (these files will be CREATED by later blocks)
snps_dir      <- file.path(proj_dir, "snps")
mask_all_snps <- file.path(snps_dir, "mask_all_snps.bed")   # all SNP positions (preferred for BSseq masking)
mask_cpg_snps <- file.path(snps_dir, "mask_cpg_snps.bed")   # CpG-only SNP positions (optional)

# Sanity prints (TRUE/FALSE is fine if masks don't exist yet)
c(
  proj_dir_exists = dir.exists(proj_dir),
  ref_exists      = file.exists(ref_fa),
  map_bt2_exists  = dir.exists(map_bt2_dir),
  snps_dir_exists = dir.exists(snps_dir),
  all_mask_exists = file.exists(mask_all_snps),
  cpg_mask_exists = file.exists(mask_cpg_snps)
)

proj_dir_exists      ref_exists  map_bt2_exists snps_dir_exists all_mask_exists 
           TRUE            TRUE            TRUE            TRUE            TRUE 
cpg_mask_exists 
           TRUE

In [ ]:
######################################################################
## BLOCK 0B — FreeBayes relaxed calling and merge (SCRIPT)
## (These steps create the cohort VCF used to build SNP masks)
######################################################################

# Per-sample (ran outside the notebook):
#   freebayes -f "$REF" sample.mkdup.bam > sample.raw.vcf 2> sample.freebayes.log
#   bgzip -f sample.raw.vcf && tabix -f -p vcf sample.raw.vcf.gz
#   # Ensure distinct sample names:
#   bcftools reheader -s <(echo "SAMPLE_NAME") -o sample.fix.vcf.gz sample.raw.vcf.gz
#   mv sample.fix.vcf.gz sample.raw.vcf.gz && tabix -f -p vcf sample.raw.vcf.gz
#
# Merge all samples:
#   cd ~/Mytilus-WGS/mapping_bt2
#   bcftools merge mc*.raw.vcf.gz -Oz -o snps/freebayes_relaxed_bt2.vcf.gz
#   tabix -f -p vcf snps/freebayes_relaxed_bt2.vcf.gz


In [ ]:
######################################################################
## BLOCK 0C — Build SNP masks from merged FreeBayes VCF (SCRIPT)
######################################################################

# From project root: cd ~/Mytilus-WGS ; conda activate bcftools-env
# Ensure file is indexed:
#   tabix -f -p vcf snps/freebayes_relaxed_bt2.vcf.gz
#
# All-SNP mask (0-based BED):
#   bcftools query -f'%CHROM\t%POS0\t%POS\n' snps/freebayes_relaxed_bt2.vcf.gz \
#   | sort -k1,1 -k2,2n \
#   | bedtools merge -i - > snps/mask_all_snps.bed
#
# CpG-only SNP mask (heuristic C>T or G>A at CpGs):
#   bcftools query -f'%CHROM\t%POS0\t%POS\t%REF\t%ALT\n' snps/freebayes_relaxed_bt2.vcf.gz \
#   | awk '($4=="C" && $5=="T") || ($4=="G" && $5=="A") {print $1"\t"$2"\t"$3}' \
#   | sort -k1,1 -k2,2n | bedtools merge -i - > snps/mask_cpg_snps.bed

In [ ]:
######################################################################
## BLOCK 0D — Partner SNP mask - Masks the CpG itself and its complementary base (SCRIPT)
######################################################################

# from ~/Mytilus-WGS
set -euo pipefail

VCF=snps/freebayes_relaxed_bt2.vcf.gz
FAI=/home/local/ADS/lsh/Mytilus/prepped_genome/mytilus_californianus_genome.fasta.fai
OUT=snps/mask_all_snps_partner.bed

# sanity: both tools available via conda
conda run -n bcftools-env bcftools --version >/dev/null
conda run -n bedtools bedtools --version >/dev/null

# 0) make sure the VCF is indexed
[[ -s "${VCF}.tbi" ]] || conda run -n bcftools-env tabix -f -p vcf "$VCF"

# 1) extract only CpG-affecting SNPs (C>T or G>A) as 1bp BED (0-based, half-open)
conda run -n bcftools-env bcftools view -v snps -H "$VCF" \
| awk 'BEGIN{OFS="\t"}{
    chr=$1; pos=$2; ref=$4; alt=$5;
    if ((ref=="C" && alt=="T") || (ref=="G" && alt=="A")) {
        print chr, pos-1, pos
    }
}' \
| sort -k1,1 -k2,2n \
| conda run -n bedtools bedtools merge -i - \
> "$OUT"

# 2) quick stats
echo "[OK] Partner-only SNP mask -> $OUT"
wc -l "$OUT" | awk '{print "BED intervals:", $1}'
awk '{bp+=$3-$2}END{print "Total masked bp:", bp}' "$OUT"

# 3) spot-check first 5 lines
head -n 5 "$OUT"

In [ ]:
######################################################################
## BLOCK 0D - SCRIPT: Merging WGBS strands (Bismark)
######################################################################

genome_folder="/home/local/ADS/lsh/Mytilus/prepped_genome/"
cd /home/local/ADS/lsh/Mytilus/mapping
eval "$(conda shell.bash hook)"
conda activate bismark

find *deduplicated.bismark.cov.gz \
| xargs basename -s deduplicated.bismark.cov.gz \
| xargs -I{} coverage2cytosine \
  --genome_folder ${genome_folder} \
  --output {}_merged.cov \
  --merge_CpG \
  --zero_based \
  {}deduplicated.bismark.cov.gz

In [2]:
######################################################################
## BLOCK 1A: GLOBAL SETUP: Paths, Libraries
######################################################################

## SNP mask to use for BSseq filtering
## Default (all SNPs): 
snp_bed_path <- "/home/local/ADS/lsh/Mytilus-WGS/snps/mask_all_snps.bed"

## If you specifically want CpG-only SNP masking, switch to:
## snp_bed_path <- "/home/local/ADS/lsh/Mytilus-WGS/snps/mask_cpg_snps.bed"

suppressMessages({
  library(bsseq)
  library(data.table)
  library(dplyr)
  library(purrr)
  library(GenomicRanges)
  library(tidyr)
  library(ggplot2)
})

In [3]:
######################################################################
## BLOCK 1B: Load and Merge .cov Files
######################################################################

cov_dir <- "/home/local/ADS/lsh/Mytilus/mapping-2/"
cov_files <- list.files(path = cov_dir, pattern = "merged_CpG_evidence\\.cov$", full.names = TRUE)
sample_names <- c("M44-9-1yr", "M45-11-5yr", "M46-14-4yr", "M47-15-6yr", "M48-18-8yr", "M49-19-8yr", "M50-24-1yr")

read_cov <- function(file, sample) {
  dt <- fread(file, header = FALSE)
  dt <- dt[, .(chr = V1, pos = V2, X = V5, N = V5 + V6)]
  setnames(dt, c("X", "N"), c(paste0("X_", sample), paste0("N_", sample)))
  return(dt)
}

cov_data <- mapply(read_cov, cov_files, sample_names, SIMPLIFY = FALSE)
combined_cov <- Reduce(function(x, y) merge(x, y, by = c("chr", "pos"), all = TRUE), cov_data)
combined_cov[is.na(combined_cov)] <- 0

In [4]:
######################################################################
## BLOCK 2: Create Raw BSseq Object and Save
######################################################################

M <- as.matrix(combined_cov[, grep("^X_", names(combined_cov)), with = FALSE])
Cov <- as.matrix(combined_cov[, grep("^N_", names(combined_cov)), with = FALSE])
colnames(M) <- colnames(Cov) <- sub("^X_", "", colnames(combined_cov)[grep("^X_", names(combined_cov))])
BSseq_obj <- BSseq(chr = combined_cov$chr, pos = combined_cov$pos, M = M, Cov = Cov, sampleNames = colnames(M))
save(BSseq_obj, file = "bsseq_raw_all_samples.RData")

In [5]:
######################################################################
## BLOCK 3: Remove WGBS Coverage Outlier
######################################################################

cov_stats <- data.frame(
  sample = sampleNames(BSseq_obj),
  covered_4x = sapply(seq_len(ncol(BSseq_obj)), function(i) sum(getCoverage(BSseq_obj, type = "Cov")[, i] >= min_meth_cov))
)

outlier_sample <- cov_stats$sample[which.min(cov_stats$covered_4x)]
BSseq_obj_clean <- BSseq_obj[, !(sampleNames(BSseq_obj) %in% outlier_sample)]
save(BSseq_obj_clean, file = "bsseq_cleaned.RData")

In [36]:
######################################################################
## BLOCK 4A: SNP Masking on outlier-removed sample set (6 samples)
######################################################################

library(rtracklayer)
library(GenomicRanges)
library(GenomeInfoDb)

# Your objects
bs   <- BSseq_obj_clean
mask <- import("/home/local/ADS/lsh/Mytilus-WGS/snps/mask_all_snps_partner.bed.gz")

# 1) Align on shared seqlevels (no style changes)
shared <- intersect(seqlevels(bs), seqlevels(mask))
length(shared)            # should be ~175 if they match now
mask   <- keepSeqlevels(mask, shared, pruning.mode = "coarse")
bs     <- keepSeqlevels(bs,   shared, pruning.mode = "coarse")

# (Optional) set the same 'genome' tag so Bioconductor stops warning
genome(mask) <- genome(bs)

# 2) Compute overlaps and filter
hits <- countOverlaps(granges(bs), mask, type = "any")
n_total   <- length(hits)
n_overlap <- sum(hits > 0)
n_keep    <- n_total - n_overlap
cat("Total loci:", n_total, "\nOverlap:", n_overlap, "\nRetained:", n_keep, "\n")

# 3) Apply the mask
BSseq_obj_masked <- bs[hits == 0, ]

# Save result
save(BSseq_obj_masked, file = "bsseq_snp_masked.RData")

[1] 174

Total loci: 27531884 
Overlap: 429099 
Retained: 27102785 


In [32]:
######################################################################
## BLOCK 4B: SNP Masking on full sample set (7 samples)
######################################################################

# Load the 7-sample BSseq object (produced before any sample removal)
load("bsseq_raw_all_samples.RData")   # loads BSseq_obj

# Make it explicit and verify sample count = 7
bs7 <- BSseq_obj
stopifnot(length(colnames(bs7)) == 7)

# Read SNP mask (padded ±1bp — CpG-safe)
library(rtracklayer)
library(GenomeInfoDb)
library(GenomicRanges)
mask <- import("/home/local/ADS/lsh/Mytilus-WGS/snps/mask_all_snps_partner.bed.gz")

# Align on shared seqlevels (no style conversions)
shared <- intersect(seqlevels(bs7), seqlevels(mask))
mask  <- keepSeqlevels(mask, shared, pruning.mode = "coarse")
bs7   <- keepSeqlevels(bs7,  shared, pruning.mode = "coarse")
genome(mask) <- genome(bs7)

# Overlap counts
hits      <- countOverlaps(granges(bs7), mask, type = "any")
n_total   <- length(hits)
n_overlap <- sum(hits > 0)
n_keep    <- n_total - n_overlap
cat("Total loci:", n_total, "\nOverlap:", n_overlap, "\nRetained:", n_keep, "\n")

# Apply the mask
BSseq_obj_masked7 <- bs7[hits == 0, ]

# Sanity checks: still 7 samples
stopifnot(length(colnames(BSseq_obj_masked7)) == 7)
cat("Samples kept (n=", length(colnames(BSseq_obj_masked7)), "): ",
    paste(colnames(BSseq_obj_masked7), collapse = ", "), "\n", sep="")

# Save
save(BSseq_obj_masked7, file = "bsseq_snp_masked_all_samples.RData")

Total loci: 27531884 
Overlap: 429099 
Retained: 27102785 
Samples kept (n=7): M44-9-1yr, M45-11-5yr, M46-14-4yr, M47-15-6yr, M48-18-8yr, M49-19-8yr, M50-24-1yr


In [33]:
############################################
## BLOCK 4C: Global mean across samples
############################################

# Extract methylation proportions (β values) per locus per sample
meth_mat <- getMeth(BSseq_obj_masked7, type = "raw")  # mCG / CG at each locus

# Average across all loci and individuals
global_mean <- mean(meth_mat, na.rm = TRUE) * 100  # convert to %
global_mean

global_per_sample <- colMeans(meth_mat, na.rm = TRUE) * 100
print(global_per_sample)

[1] 11.48458

 M44-9-1yr M45-11-5yr M46-14-4yr M47-15-6yr M48-18-8yr M49-19-8yr M50-24-1yr 
 12.503574  13.848951  12.718962  11.325192  10.297695  10.004323   9.951701 
